# Differential Privacy and its Interpretations
:label:`dp_definition`




In the last notebook, we learned about the privacy challenges we face and why it is not a straightforward problem to address. Specifically, we established that none of the following seemingly intuitive approaches to privacy protection really works:
* Anonymized datasets with PII removed
* Database system that returns answers only if it is truly aggregates
* Opaqueness in Machine learning models

These approaches are easily broken under data linkage attack, differencing attack, membership inference attack, as well as data extraction / reconstruction attacks. The wide availability of additional information about individuals in the age of Internet has made it much easier to carry out some of these attacks in practice and in large scale. 

These examples highlight the need for a more rigorous approach to privacy protection that provably addresses these challenges. The computer science approach for solving such problems is to first write down a formal mathematical definition that describes "privacy", then to come up with algorithms that come with a mathematical proof that the definition is satisfied. Differential privacy is one such definition. But before we formally define differential privacy, let us take an axiomatic approach by first articulating what any reasonable privacy definition should satisfy.

## The principle of reasonable privacy definitions

**(Remove this paragraph or defer to later)** The first decision to make is whether "privacy" should be the property of a dataset or a computation performed on the dataset. The former makes sense if we would like to publish a sanitized dataset, but cannot describe database query or machine learning jobs on the dataset in which the output is not a dataset. The latter is strictly more general. To illustrate the subtle differences, it is the procedure to remove PII, rather than the output of the PII-removed dataset that we should define privacy on; and it is the algorithm that database system use to respond to SQL queries, rather than the numbers that they return.

One could argue that any reasonable definition of the privacy (of a computation) should satisfy the following criteria:

>1. It prevents (or at least mitigates) most (if not all) attacks known to date
>2. It does not make strong assumptions about the attacker
>3. It does not make strong assumptions about the input data
>4. It should not break under postprocessing, mixing and composition.

The first three criteria are straightforward. We want the definition to be effective in addressing the privacy challenges. We do not wish to need to model the attackers or the data because otherwise it restricts the protection to those cases where these assumptions are true. It is generally difficult to check the validity of these assumptions due to the abundance of side information and the computational resources available.

The last point requires some explanation. Let's say we compute $Y$ as a function of the input dataset $D$ and $Y$ satisfies a privacy definition (in short, we just say $Y$ is *private*), then it will be totally unacceptable if there is a function $f$ such that $f(Y)$ does not satisfy $P$. Similarly, if $Y_1,Y_2$ are two functions of the data, *mixing* says that if $Y_3$ is defined by returning $Y_1$ with probability $p$ and $Y_2$ with probability $1-p$, then $Y_3$ should not be less private than both $Y_1$ and $Y_2$.  In addition, if $Y_4$ returns the concatenation $(Y_1,Y_2)$, known as *composition*, then $Y_4$ should still be private to some extent, even though it is well-expected that it must be less private than $Y_1$ and $Y_2$ individually. 
 

 
## Examples of bad privacy definitions

These general criteria can be used to easily weed out bad privacy definitions. To illustrate this point, let us define a few *strawman* definitions and inspect them by the principles we specified earlier.

> **Definition 1 (Non-identifying)** We say a data anonymization procedure $M$ is private if for any dataset $D$,  $M(D)$ returns a dataset $\tilde{D}$ such that the *information* in $\tilde{D}$ is insufficient for identifying any individuals (PIIs and pseudo-identifiers are all removed).  

We already know that this definition is not robust to data-linkage attack so it fails the second rule from our Netflix prize example. But let's give it the benefit of doubt and argue that the attacker does *not* have any side information. In this case, it seems good enough if the attacker will always have some ambiguity due to the missing information.  However, the following example shows that it fails completely under composition. 

Let's say $M_1$ outputs dataset $\tilde{D}_1$ which contains the first four digits of my social security number and $M_2$ outputs outputs a dataset $\tilde{D}_2$ which contains the last three digits.  Clearly, neither $M_1$ or $M_2$ by themself contains sufficient information for reconstructing my SSN, the composition of them clearly does. 


> **Definition 2 ($k$-aggregate)**  We say that an algorithm $M$ that answers counting queries (e.g., \# of people who has heart disease) is $k$-aggregates-private if it returns the correction answer if the answer is larger than $k$ and return nothing otherwise. 

Clearly, the differencing attack that we described earlier can be thought of as the composition of two $k$-aggregate algorithms with $k$ in the thousands. But taking the difference of them --- a form of post-processing --- would lead to a catastrophic failure that reduces $k$ to $1$ and it  directly identifies information about a single person.

While the above two definitions, when satisfied, do provide some privacy benefits on the intuition level, they are not robust and fail on one or more basic principles behind any *good* privacy definitions. It is surprisingly tricky to come up with a good definition for privacy. There are researchers who argue that any privacy definition that satisfies these axioms will have to look very much like *differential privacy*, which I will formally introduce next. 

As a side note, intuitions like those in Definition 1 and 2 above still do matter and sometimes they can be formally quantified. If you manage to read on and reach the last section of the chapter, you will see some pointers to recent research articles on on how some of these approaches such as masking all $0$ or near $0$s in frequency tables and removing PIIs  can be leveraged to design stronger differentially private algorithms.






## Differential privacy


![Illustration of the two indistinguishable worlds in differential privacy.](imgs/dp1.jpg)
:label:`fig_dp_illus`

<span style="color:blue">TODO: Figure from my DP tutorial with Borja. Need to remake... </span>.

Differential privacy is defined through a cryptographic notion called **indistinguishability**, which talks about two distributions being almost identical to each other so that a sample from one of the distributions cannot be used infer which distribution it was sampled from. 

> **Definition (Indistinguishability)**  Let $P,Q$ be two distributions. We say that $P,Q$ are $(\epsilon,\delta)$-Indistinguishable if for any possible event $E$ in the output space,
\begin{align*}
P(E) \leq e^\epsilon Q(E) + \delta, \\
Q(E) \leq e^\epsilon P(E) + \delta.
\end{align*}
We denote the above by $P\approx_{\epsilon,\delta} Q$. 

Indistinguishability implies that it is information-theoretically hard for any inference algorithm (same as a binary classifier for machine learners, or a hypothesis test for statisticians) to confidently determine whether a sample is coming from $P$ or $Q$.   $\epsilon,\delta$ are parameters tha measures how similar $P$ and $Q$ are in multiplicative and additive distances respectively.  


> **Definition (Neighbors)** Two datasets $D,D'$ are neighbors if $D'$ for a distance function $d$, $d(D,D')\leq 1$.

Two common neighboring relationships are the **Add/Remove** neighbors and the  **replace-one** neighbor.  The distance function $d(D,D')$ under the **Add/Remove** neighboring relationship is the number of data points to add or remove before we can modify $D$ into $D'$. So we can add Alice a dataset $D$ or remove Alice from the dataset $D$ (if Alice is already in $D$) to create the neighboring dataset $D'$.

In the **replace-one** neighboring relationship, the $D$ and $D'$ have the same size $n$, i.e., the number of individuals. The distance function $d(D,D')$ measures the number of data points we need to *replace* to convert $D$ to $D'$, i.e., Hamming Distance. For example, by replacing Alice from $D$ with Bob, we obtain a neighbor dataset $D'$. 

The *Add/Remove* neighboring relationship is slightly more general than the *replace-one* neighbor, thus it is often the preferred neighboring relationship for defining DP. However, the *replace-one*  neighbor version of DP is also popular (and technically convenient in some cases). 

> **Definition (Differential Privacy)** Let $\mathcal{M}$ be a *randomized algorithm* (referred to as a *mechanism* traditionally). We say that $\mathcal{M}$ is $(\epsilon,\delta)$-differentially private (or $(\epsilon,\delta)$-DP) if for any pair of neighboring dataset $D,D'$, the two distributions $\mathcal{M}(D)\approx_{\epsilon,\delta} \mathcal{M}(D')$.


Intuitively, by the definition of indistinguishbability, DP says that the presence and absence of any individual cannot be infered using the output of the algorithm $\mathcal{M}$. $\epsilon,\delta$ are known as privacy parameters. The smaller they are the stronger the privacy guarantee is that is provided by this definition.
When $\delta = 0$ we say $\mathcal{M}$ satisfies $\epsilon$-*pure* differential privacy, and when $\delta > 0$, it satisfies *approximate* differential privacy. 

This idea is illustrated in Figure (:numref:`fig_dp_illus`), where algorithm $\mathcal{M}$ obfuscates the difference between the two parallel worlds via randomization.

Note that DP is a property of an algorithm $\mathcal{M}$ alone. It does not make unverifiable distributional assumptions on how the data are generated, thus universally applicable to any input dataset and protects against the inference about any individual (in the dataset or not).

Another often confusing aspect of DP is that *it is only a definition*. It does *not* directly concern how to design algorithms that satisfy this definition. This will be the topic of the next notebook.




## "The promise of differential privacy"

Differential privacy guarantees can be interpreted in several different ways. It is crucial for us to understand what what it promises and how it satisfies the principles of reasonable privacy definition that we started this notebook with. We will consider three interpretations of DP:

1. Bounded Risk Multiplier via the definition itself
2. Bounded information-gain in Bayesian interpretation 
3. Hypothesis testing interpretation


### Interpretation 1:  Bounded Risk Multiplier of any Bad Event
The first interpretation can be developed by staring at the definition of DP.  Let $E$ be any bad event that may happen to Bob. Even $E$ could be, e.g., Bob being successfully identified via a membership inference attack or others of your favorite bad things that could happen to Bob, such as his secret of enjoying Pineapple Pizza being discovered by his colleague. DP ensures that the probability that event $E$ when Bob is part of the dataset is not going to be much larger than that when Bob is not. For this reason, we can call $e^\epsilon$ a *risk multiplier*. 

One good application os this is to see how differential privacy prevents verbatim generation of a unique sentence in the training data of a language model (LM).  Let Bob's SSN be $123-45-678$ and let 

> `''Bob's SSN is 123-45-678.''`

be a sentence in the training data.  In this case, we can instantiate $E$ to be that the language model generates `... 123-45-678` when a prompt of `Bob's SSN is ...` is given. Then if the LM is trained under an $(\epsilon,\delta)$-DP constraint, then the probability that $E$ happens will be at most as large as $e^\epsilon P_{\text{Baseline}} + \delta$ where $P_{\text{Baseline}}$ is the probability of the LM generating the exact sequence out of nowhere when the above datapoint is not used for training. Most people would agree that even if the language model knows that it is an SSN that needs to be generated here, the probability of correctly generating all 9 digits correctly is at most $1e-9$, thus the probability of $E$ is not going to be much larger.  We emphasize that even if $\epsilon$ is chosen to be a somewhat larger value, e.g., $10$, then the bound of $1e-9 \cdot e^{10} \approx 2e-5$ --- a very reasonable level of protection against unintended memorization. 


### Interpretation 2:  Bounded information gain in Bayesian Inference about an individual

The second interpretation considers a strong adversary, Eve, who knows all but Bob's data in a dataset $D$. Let's denote it by $D_{-\text{Bob}}$. Even also possesses some auxilliary information $\textbf{aux}$ and would like to make an inference about Bob on, e.g., whether Bob likes pineapple pizza. Let's say Eve is a Bayesian and has a *prior belief*  $P(\text{Bob likes pineapple pizza.} | \textsf{aux}, D_{-\text{Bob}})$. The natural and optimal thing that Eve can do, given the new information is to update her prior belief into  a posterior belief $P(\text{Bob likes pineapple pizza.} | \textsf{aux}, D_{-\text{Bob}}, \mathcal{M}(D))$. 

Differential privacy ensures that the posterior is not much more informative than the prior.  This is best illustrated under the pure-DP with replace-one neighbor (note that $\epsilon$-DP for add/remove implies $2\epsilon$-DP for replace one). 

$(\epsilon,0)$-DP guarantee under replace-one neighbor of $\mathcal{M}$ implies that:
> $$
P(\text{Bob likes pineapple pizza.} | \textsf{aux} , D_{-\text{Bob}}) \leq e^{\epsilon} P(\text{Bob likes pineapple pizza.} |  \mathcal{M}(D), \textsf{aux}, D_{-\text{Bob}})
$$

It might be revealing to see a proof. As a short-hand, we will denote $\widetilde{\textsf{aux}}:= (\textsf{aux}, D_{-\text{Bob}})$. Denote `"Bob likes pineapple pizza."` by a Boolean variable $\mathsf{P_1}$ as a short hand, let $\neg\mathsf{P_1}$ be its complement. Let Bob be a data point in the dataset $D$. 

By Bayes rule
$$
\begin{aligned}
P[ \mathsf{P_1} | \widetilde{\textsf{aux}}, \mathcal{M}(D) ] &= \frac{P[ \mathsf{P_1} | \widetilde{\textsf{aux}}] P[\mathcal{M}(D)| \widetilde{\textsf{aux}}, \mathsf{P_1} ]}{P[ \mathsf{P_1}| \widetilde{\textsf{aux}}] P[\mathcal{M}(D)| \widetilde{\textsf{aux}},  \mathsf{P_1} ] + P[ \neg\mathsf{P_1} | \widetilde{\textsf{aux}}] P[\mathcal{M}(D)| \widetilde{\textsf{aux}},  \neg\mathsf{P_1} ]}\\
&\leq \frac{P[ \mathsf{P_1} | \widetilde{\textsf{aux}}] P[\mathcal{M}(D)| \widetilde{\textsf{aux}}, \mathsf{P_1} ]}{P[ \mathsf{P_1}| \widetilde{\textsf{aux}}] P[\mathcal{M}(D)| \widetilde{\textsf{aux}},  \mathsf{P_1} ] + P[ \neg\mathsf{P_1} | \widetilde{\textsf{aux}}] \color{blue}{e^{-\epsilon}}P[\mathcal{M}(D)| \widetilde{\textsf{aux}},  \color{blue}{\mathsf{P_1}} ]}\\
&\leq  \frac{P[ \mathsf{P_1} | \widetilde{\textsf{aux}}] \cdot P[\mathcal{M}(D)| \widetilde{\textsf{aux}}, \mathsf{P_1} ]}{  \color{blue}{e^{-\epsilon}} \cdot  P[\mathcal{M}(D)| \widetilde{\textsf{aux}}, \mathsf{P_1} ]}\\
&= e^{\epsilon} P[ \mathsf{P_1} | \widetilde{\textsf{aux}}].
\end{aligned}
$$
where in the second line, we applied the definition of differential privacy, by observing that by replacing $\neg\mathsf{P_1}$ with $\mathsf{P_1}$ we create a neighboring dataset that differs only in Bob's pizza preference.

Note that this is a very strong guarantee that says that no matter what auxillary information the adversary has, no matter what inference she is trying to conduct about an individual Bob, and no matter what the randomized algorithm $\mathcal{M}(D)$ ends up realizing into, Eve won't be much more successful relative to not seeing the output $\mathcal{M}(D)$ at all. 

Observe that this suggests pure differential privacy provides strong relative guarantees again any side information that might be available. We did not talk about approximate DP yet.  When $\delta>0$, the protection of such type is slight weaker and more complex to explain, so we will not go into the details. But it also a relative guarantee that works universally for all datasets and any $\textsf{aux}$.

**What if Eve does not know know $D_{-\text{Bob}}$?** This is a subtle question and because Eve is able to learn something new about the distribution of $D_{-\text{Bob}}$ when updating her belief into a posterior.  Let us assume that Eve has a believe  $D_{-\text{Bob}} \sim \mu(\cdot)$.   In this case, the prior and posterior belief of Eve are $P[ \mathsf{P_1} | \textsf{aux}]$ and $P[ \mathsf{P_1} | \textsf{aux}, \mathcal{M}(D) ] $, which can be calculated by marginalizing over $D_{-\text{Bob}}\sim \mu$ and $D_{-\text{Bob}}\sim \mu(\cdot|\mathcal{M}(D) )$ respectively.

It follows that
$$
P[ \mathsf{P_1} | \textsf{aux}, \mathcal{M}(D) ]  = \mathbb{E}_{D_{-\text{Bob}}\sim \mu(\cdot|\mathcal{M}(D) )}[ P[ \mathsf{P_1} | \widetilde{\textsf{aux}}, \mathcal{M}(D) ]]  \leq e^{\epsilon} \mathbb{E}_{D_{-\text{Bob}}\sim \mu(\cdot|\mathcal{M}(D) )}[  P[ \mathsf{P_1} | \widetilde{\textsf{aux}}] ] \leq  e^{\epsilon} \max_{D_{-\text{Bob}}} \left[\frac{\mu(D_{-\text{Bob}}|\mathcal{M}(D) )}{\mu(D_{-\text{Bob}})}\right]  P[ \mathsf{P_1} | \textsf{aux}],
$$

There are two parts of the information gain. The first part $\max_{D_{-\text{Bob}}} \left[\frac{\mu(D_{-\text{Bob}}|\mathcal{M}(D) )}{\mu(D_{-\text{Bob}})}\right]$ is due to how much Eve learns about the dataset as a whole, and the second part, specific to Bob, remains bounded by $e^{\epsilon}$. 

When Eve does not know much about $D_{-\text{Bob}}$ a priori. It is possible that knowing $\mathcal{M}(D)$ makes the auxiliary information $\textsf{aux}$ more useful, so it changes Eve's belief substantially (see the Terry Gross example towards the end of this notebook). But the above derivation shows that bulk of the change is coming from the extent to which $\mathcal{M}(D)$ changes the belief about $D_{-\text{Bob}}$ rather than the belief about Bob alone. 


### Interpretation 3:  Hypothesis testing view
Finally, DP can be interpreted a bound on how much an attacker can do, which gives rise to an intuitive alternative definition that makes sense to everyone understanding binary classification.  Consider an adversary who tries to distinguish between two neighboring datasets $D$ and $D’$. Think about this as a binary classier (or a hypothesis test), whose performance is measured by a trade-off function between the Type I error (False positive Rate) and Type II error (False Negative Rate). Saying a mechanism is $(\epsilon,\delta)$-DP is equivalent to say that all adversary must operates above the solid line of Type I error-Type II error tradeoff function, shown in Figure (:numref:`fig_dp_alternative_def`), i.e., no classifiers can be accurate.  In particular, it implies that the AU-ROC is close to 0.5 when the privacy parameters are small.   Recall that the ROC curve plots (1- Type II error) against Type I error; thus is exactly the up-side-down version of Figure (:numref:`fig_dp_alternative_def`) below.

![The hypothesis testing interpreation of DP.](imgs/tradeoff_function.png)
:label:`fig_dp_alternative_def`

<span style="color:blue">TODO: Figure quoted from an existing paper.. need to remake, and highlight impossibility region</span>.

The hypothesis testing view of DP implies lower bounds on all popular metrics for classification problems such as AUC and F1 Score, thus directly compatible with the empirically reported numbers under these metrics via membership inference attacks. For this reason, DP can be viewed as providing certified robustness towards membership inference attacks.


### DP and the principles of reasonable privacy definitions
From the interpretations above, it is clear that DP satisfies Principle 2 and 3 about not making strong assumptions on either the adversary or the data. We will see how it satisfies composition and postprocessing principles in future notebooks.




## What DP does not prevent.


So far we focused on what DP protects against. It is also crucial to for us to understand what privacy risks DP is not preventing against and why in some sense some of these "privacy risks" are directly at odds with the utility of a data analysis.  

To start, we use a classical example to illustrate that DP does not prevent all possible harms that could happen to an individual due to the data analysis it enables. 

> **Example: Smoking Causes Cancer.**   Bob is a smoker. Bob decides to participate in a study to reveal whether smoking causes cancer, which conducts the study differentially privately. The results of the study conclusively show that smoking does causes cancer. The report from the study is read by Bob's insurance company, who decides to increase the premium for all smokers.  Bob's insurance premium increases. Does this contradicts DP?

Note that Bob is genuinely harmed financially due to the discovery of the fact "Smoking causes cancer", but Bob will be harmed in the same way regardless of Bob's participation. As a matter of fact, "Smoking causes cancer" is a statistical fact that is the data analysis is hoping to discover. It is about the distribution rather than a specific dataset sampled from the distribution. In fact, one could argue that any privacy-perserving techniques that prevent such data analysis from discovering or validating a distribution-level fact are failing the very purpose of conducting the data analysis in the first place.

Ther power of differential privacy is that it disentangles the harm from learning the distribution (or the population), and the harm due to an individual's participation in a dataset.  DP provably prevents the second type of harm but tries its best to not imposing any restrictions on the first type. 

We emphasize that the problem of restricting certain population-level information from being learned from a sample drawn from that population is an important problem too. It is just not the problem DP addresses. Instead, there is a large body of work on Inferential Privacy, e.g., Pufferfish privacy (and also Algorithmic Fairness) that aims at formulating and solving versions of this problem.


Let's consider another classical example on what DP does not prevent and why it is fine.
> **Example: Terry Gross's height.** An attacker has access to an auxiliary information that "Terry Gross is two inches shorter than an average Lithuanian woman". The average height of a Lithuanian woman is released, with differential privacy, and by combining with the auxiliary information, the attacker now knows Terry Gross's height, as accurately as the extent the average height can be released under DP. 

Did we not promise that DP is robust against any side information $\textsf{aux}$ is known by the attacker? It seems paradoxy because if the attacker's prior belief of the average height of a Lithuanian woman is a uniform distribution between 4 feet and 6 feet, then 
$$
P[ \text{Terry Gross's height} | \textsf{aux} ]
$$
is a uniform distribution between 4 feet 2 inch and 6 feet 2 inch.
Then
$$
P[ \text{Terry Gross's height} | \textsf{aux},  \mathcal{M}(D) = \text{5 feet} ]
$$
under a constant DP bound, ensures that the posterior belief concentrates around 5 feet 2 inch plus and minus $O(\frac{1}{\sqrt{n}} + \frac{1}{n\epsilon})$ where $\epsilon$ is the privacy loss parameter and $n$ is the number of data points in a dataset of iid sampled Lithuanian women with high probability.  For this reason, the information gain can be very large in this case. 

We make two observations. First, this information gain will be present whether or not Terry Gross is part of $D$.  Second, no mechanism $\mathcal{M}$ that accurately reports the average height can be used to prevent such information gain. 

We can further decompose the information gain as follows 

$$
\frac{P[ \text{Terry Gross's height} | \textsf{aux},  \mathcal{M}(D) ]}{P[ \text{Terry Gross's height} | \textsf{aux} ]} = \underbrace{\frac{P[ \text{Terry Gross's height} | \textsf{aux},  \mathcal{M}(D) ]}{ P[ \text{Terry Gross's height} | \textsf{aux},  \mathcal{M}(D_{-\text{Terry}})]}}_{\leq e^\epsilon\text{ due to DP.}} \underbrace{\frac{P[ \text{Terry Gross's height} | \textsf{aux},  \mathcal{M}(D_{-\text{Terry}})]}{P[ \text{Terry Gross's height} | \textsf{aux} ]}}_{\text{Needed even if Terry does not participate.}}.
$$

The first part of the ratio can be shown to be bounded by $e^\epsilon$ thanks to DP, while the second part captures the part of information-gain from learning the distribution, which will be there even if Terry's data is not used. 




Lastly, we will revisit the example of unintended memorization which we have seen earlier.

> **Example: Unintended memorization, revisited**. One of the applications of DP in ML is to prevent a language model from memorizing and generating specific text in the training data verbatim.  However, DP by itself does not provide protection to *all* text in the data. For example, if Bob's email address appears in thousands of data samples, then it is likely for Bob's email to be memorized despite DP. 

This is not suprising because as the number of data points involving a piece of information becomes large, it becomes closer to a population-level information that remains insensitive to the adding / removing of specific data point. By design, DP will allow this information to largely pass through. 


In order to DP to be effective in protecting secrets that appear many times in the data set, preprocessing steps such as deduplication and redaction are needed (see, e.g., [Zhao, Li and Wang, 2022]). More generally, there should be a clear statement of the threat model to tell whether DP is the right hammer for an application. 

 